# Projects (30 pts):

Suppose $S_1(t)$ and $S_2(t)$ are independent stock processes with equal volatility. $S_1(0)=S_2(0)=100.$,
The contract at maturity pays the value of the most expensive stock: $V(T)=max_i\{S_1(T),S_2(T)\}.$, 

* What is the formula for the fair value of the contract, $V(0)$?,
* Derive the partial differential equation for this contract,
* Solve the resultant PDE using Finite Difference Method,
* Compare the results obtained through solving the PDE with Monte Carlo Method  

# 1. Prices under Q measure:

$$ dS_i = r S_i dt + \sigma S_i dW_i, \quad i=1,2 $$

# 2. Option value at maturity:
$$ V(S_1, S_2, T) = \max(S_1, S_2) $$

# 3. Itô-formulae:
\begin{equation}
dV = \left( \frac{\partial V}{\partial t} + r S_1 \frac{\partial V}{\partial S_1} + r S_2 \frac{\partial V}{\partial S_2} + \frac{1}{2} \sigma^2 S_1^2 \frac{\partial^2 V}{\partial S_1^2} + \frac{1}{2} \sigma^2 S_2^2 \frac{\partial^2 V}{\partial S_2^2} \right) dt
\end{equation}
\begin{equation}
+ \sigma S_1 \frac{\partial V}{\partial S_1} dW_1 + \sigma S_2 \frac{\partial V}{\partial S_2} dW_2.
\end{equation}

# 4. Using martingale property:
The discounted price process of the derivative must be a matringale, this yields:
\begin{equation}
\frac{\partial V}{\partial t} + r S_1 \frac{\partial V}{\partial S_1} + r S_2 \frac{\partial V}{\partial S_2} + \frac{1}{2} \sigma^2 S_1^2 \frac{\partial^2 V}{\partial S_1^2} + \frac{1}{2} \sigma^2 S_2^2 \frac{\partial^2 V}{\partial S_2^2} = rV.
\end{equation}


# Explicit method

We want to use explicit method as it is numerically less expensive and we have a 3D problem now. For that we apply backward difference for axis "t", and central difference for stocks. Let's assume we are at point $n+1$ in time.

# $\partial V / \partial t$ backward

\begin{equation}
\frac{\partial V}{\partial t} \approx \frac{V^{n+1}_{i,j} - V^{n}_{i,j}}{\Delta t}
\end{equation}

# $\partial V / \partial S_i$ central

\begin{equation}
\frac{\partial V}{\partial S_1} \approx \frac{V^{n+1}_{i+1,j} - V^{n+1}_{i-1,j}}{2\Delta S}
\end{equation}

\begin{equation}
\frac{\partial V}{\partial S_2} \approx \frac{V^{n+1}_{i,j+1} - V^{n+1}_{i,j-1}}{2\Delta S}
\end{equation}

# $\partial^2 V / \partial S_i^2$ central
\begin{equation}
\frac{\partial^2 V}{\partial S_1^2} \approx \frac{V^{n+1}_{i+1,j} - 2V^{n+1}_{i,j} + V^{n+1}_{i-1,j}}{\Delta S^2}
\end{equation}

\begin{equation}
\frac{\partial^2 V}{\partial S_2^2} \approx \frac{V^{n+1}_{i,j+1} - 2V^{n+1}_{i,j} + V^{n+1}_{i,j-1}}{\Delta S^2}
\end{equation}


# Substituting back to PDE:

\begin{equation}
\frac{V^{n+1}_{i,j} - V^{n}_{i,j}}{\Delta t} + r S_1 \frac{V^{n+1}_{i+1,j} - V^{n+1}_{i-1,j}}{2\Delta S} + r S_2 \frac{V^{n+1}_{i,j+1} - V^{n+1}_{i,j-1}}{2\Delta S} + 
\end{equation}
\begin{equation}
+ \frac{1}{2} \sigma^2 S_1^2 \frac{V^{n+1}_{i+1,j} - 2V^{n+1}_{i,j} + V^{n+1}_{i-1,j}}{\Delta S^2} + \frac{1}{2} \sigma^2 S_2^2 \frac{V^{n+1}_{i,j+1} - 2V^{n+1}_{i,j} + V^{n+1}_{i,j-1}}{\Delta S^2} = rV^{n+1}_{i,j}
\end{equation}

as we see there is only one variable with time "n+1". Solving for $V^n_{i,j}$ yields

\begin{equation}
V^n_{i,j} = V^{n+1}_{i,j} - \Delta t \left[ r S_1 \frac{V^{n+1}_{i+1,j} - V^{n+1}_{i-1,j}}{2\Delta S} + r S_2 \frac{V^{n+1}_{i,j+1} - V^{n+1}_{i,j-1}}{2\Delta S} + \right.
\end{equation}
\begin{equation}
\left. \frac{1}{2} \sigma^2 S_1^2 \frac{V^{n+1}_{i+1,j} - 2V^{n+1}_{i,j} + V^{n+1}_{i-1,j}}{\Delta S^2} + \frac{1}{2} \sigma^2 S_2^2 \frac{V^{n+1}_{i,j+1} - 2V^{n+1}_{i,j} + V^{n+1}_{i,j-1}}{\Delta S^2} - r V^{n+1}_{i,j} \right]
\end{equation}

We can iterate through time backwards and with a simple for cycle calculate these values from the previous layer.

Chosen parameters:
- r=5%
- $\sigma$ = 20%
- T=7/365 (one week)

We need to choose discretization parameters so that stability holds.

In [1]:
from typing import Callable
import numpy as np
import matplotlib.pyplot as plt

In [70]:
def initialize_grid(M, N, T, S1,S2, r, sigma):
    dS = 2*max(S2,S2) / (M-1)
    S_max=2*max(S2,S2)
    dt = T / (N-1)
    V = np.zeros((N, M, M))
    
    S1_grid = np.linspace(0, S_max, M)
    S2_grid = np.linspace(0, S_max, M)
    t_grid = np.linspace(0, T, N)
    
    for i in range(N):
        for j in range(M):
            V[i, 0,  j] = S2_grid[j]
            V[i, -1, j] = S1_grid[-1]
            V[i, j,  0] = S1_grid[j]
            V[i, j, -1] = S2_grid[-1]
    
    for i in range(M):
        for j in range(M):
            V[-1, i, j] = max(S1_grid[i], S2_grid[j])
    
    return (V, dS, dt, r, sigma,S1,S2)

V = initialize_grid(50, 20000, 7/365, 100,100, 0.05, 0.2)

In [71]:
def solve_pde(gridpack):
    grid=gridpack[0]
    dims=grid.shape
    N=dims[0]
    M=dims[1]
    delta_t=gridpack[2]
    delta_S=gridpack[1]
    S1=grid[-1, :, 0]
    S2=grid[-1, 0, :]
    #print(S1)
    #print(S2)
    S1now=gridpack[5]
    S2now=gridpack[6]
    r=gridpack[3]
    sigma=gridpack[4]
    for i in range(N-2, -1, -1):
        for j in range(1, M-1):
            for k in range(1, M-1):
                grid[i, j, k] = max(0, grid[i+1, j, k] - delta_t * ( \
                r * S1[j] * (grid[i+1, j+1, k] - grid[i+1, j-1, k]) / (2 * delta_S) + \
                r * S2[k] * (grid[i+1, j, k+1] - grid[i+1, j, k-1]) / (2 * delta_S) + \
                0.5 * sigma**2 * S1[j]**2 * (grid[i+1, j+1, k] - 2*grid[i+1, j, k] + grid[i+1, j-1, k]) / (delta_S**2) + \
                0.5 * sigma**2 * S2[k]**2 * (grid[i+1, j, k+1] - 2*grid[i+1, j, k] + grid[i+1, j, k-1]) / (delta_S**2) - \
                r * grid[i+1, j, k] ))
    
    #return grid
    # interpolation for initial S1, S2
    j = np.searchsorted(S1, S1now) - 1
    k = np.searchsorted(S2, S2now) - 1

    j = max(1, min(len(S1)-2, j))
    k = max(1, min(len(S2)-2, k)) 
    V11 = grid[0, j, k]
    V12 = grid[0, j, k+1]
    V21 = grid[0, j+1, k]
    V22 = grid[0, j+1, k+1]

    V1 = V11 + (S1now - S1[j]) / (S1[j+1] - S1[j]) * (V21 - V11)
    V2 = V12 + (S1now - S1[j]) / (S1[j+1] - S1[j]) * (V22 - V12)

    value = V1 + (S2now - S2[k]) / (S2[k+1] - S2[k]) * (V2 - V1)

    return value


In [72]:
res=solve_pde(V)
res

99.76690907818923

# Monte Carlo benchmark

In [56]:
def monte_carlo(S1, S2, T, sig1, sig2, r, n=1000, conf=False):
    corr=0
    N1=np.random.normal(0, 1, n)
    N2=np.random.normal(0, 1, n)
    B1=N1
    B2=N1*corr+np.sqrt(1-corr**2)*N2
    ST1=S1*np.exp((r-0.5*sig1**2)*T+sig1*B1*np.sqrt(T))
    ST2=S2*np.exp((r-0.5*sig2**2)*T+sig2*B2*np.sqrt(T))

    value_T=np.maximum(ST1,ST2)

    price=np.mean(value_T)*np.exp(-r*T)
    if conf==False:
        return price
    else:
        std_error=np.std(value_T)*np.exp(-r*T)/np.sqrt(n)
        inf=[price-1.96*std_error, price+1.96*std_error]
        return [price, *inf]

In [69]:
monte_carlo(100,100,7/365,0.2,0.2,0.05)

101.65286846117336

The price is close for the given parameters. The asset price discretization should be denser, but for that N needs to be increased to achieve stability. 